<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!python -m pip install --upgrade pip setuptools wheel
!pip install libtorrent lbry-libtorrent dropbox tomlkit

!curl https://rclone.org/install.sh | sudo bash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 55.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 19.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [dropbox]
  % Total    

# Imports
rich.progress for concurrent progress tracking<br>
libtorrent as torrent client<br>
rclone as the cloud interface<br>

In [2]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed

import logging
import time
from timeit import default_timer as timer

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
from pathlib import Path

from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

import libtorrent as lt

# Setup Filesystem

In [3]:
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("torrents stored in a folder or are a single file"))
  magnet_document.add("show1fldr", example_folder_tor)
  magnet_document.add("movie1", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("show2fldr", example_multifiles)

  magnet_file.write(magnet_document)

# Download Torrents

In [4]:
# TODO: Handle torrent files
def download_torrent(progress: Progress, media_name: str, link: str):
  MAX_FILENAME_LEN = 35     # rich progress constant
  METADATA_TIMEOUT_MS = .8

  # generate save path for torrent download
  save_path=Path("/content/Torrents/") / media_name
  if not save_path.exists():
    save_path.mkdir(parents=True)

  # setup rich progress
  task = progress.add_task(
        "download",
        total=100,
        status="[yellow]Obtaining Metadata",
        filename=media_name,
        speed="0.0 kB/s",
  )
  ses = lt.session()

  # setup torrent handle
  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    progress.update(task, status="[red]Failed to parse magnet!")
    return None
  atp.save_path = str(save_path.as_posix())

  # generate the metadata to download torrents
  handle = ses.add_torrent(atp)
  status = handle.status()

  while not handle.status().has_metadata:
    time.sleep(METADATA_TIMEOUT_MS/1000)
    status = handle.status()

  filename =  status.name if len(status.name) < MAX_FILENAME_LEN \
              else f"{status.name[:MAX_FILENAME_LEN]}..."

  # start progress bar
  progress.update(task,
                  status="[yellow]Downloading",
                  filename=filename
  )

  # proceed to download the torrent
  while not status.is_seeding:
    status = handle.status()

    progress.update(task,
                completed=status.progress * 100,
                speed=f"{status.download_rate/1000:.1f} kB/s",
    )

  # finished status
  progress.update(task,
                  status="[green]Complete",
                  completed=100,
  )

  return save_path

In [ ]:
with Progress(
  TextColumn("{task.fields[status]}"),
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
) as progress:

  # multithread thread downloading
  with ThreadPoolExecutor() as executor:
    media = magnet_file.read()
    futures = []
    for m in media:
      magnet_links = media[m]["magnets"]

      for link in magnet_links:
        futures.append(executor.submit(download_torrent, progress, m, link))

    for future in as_completed(futures):
      try:
          future.result()
      except Exception as e:
          print("Upload failed:", e)

Output()

# Upload to Dropbox
If this is the first time you are using this program, install rclone on your local machine from this [link](https://rclone.org/install/) and run ```rclone authorize dropbox```. After following the prompts to allow access to your dropbox, paste the result from the terminal when prompted for the token.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from configparser import ConfigParser
from pathlib import Path

cfg = ConfigParser()

# observed that google has this file and copy that config file
gdrive_savepath = Path("/content/drive/MyDrive/Apps/torrent_dl/rclone/rclone.conf")
rclone_path = Path("/root/.config/rclone/rclone.conf")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# check if cloud config file exists
if gdrive_savepath.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  !cp {gdrive_savepath.as_posix()} {rclone_path.as_posix()}

# create the config file if it doesn't exist in the cloud or locally
if not rclone_path.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  rclone_path.touch()

  cfg['dropbox'] =  \
  {
      "type": "dropbox",
      "token": ""
  }

  # get refresh token from user host
  token = input("With rclone installed on a local system outside of google colab, run \"rclone authorize dropbox\" and paste the result here: ")
  cfg['dropbox']['token'] = token

  # generate the rclone config file
  with rclone_path.open('w') as cfgwriter:
    cfg.write(cfgwriter)

In [ ]:
!rclone copy ./Torrents/ dropbox:/Anime/

In [ ]:
# save config to google drive for future use
if not gdrive_savepath.exists():
  gdrive_savepath.parent.mkdir(parents=True, exist_ok=True)
  gdrive_savepath.touch()

!cp {rclone_path.as_posix()} {gdrive_savepath.as_posix()}